In [9]:
!pip install elasticsearch

In [10]:
from datetime import datetime
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "p2iFCHUbC7ze1QoIMVw"

es = Elasticsearch("http://elasticsearch:9200",
                    basic_auth=("elastic", ELASTIC_PASSWORD))

es.info()

ObjectApiResponse({'name': 'es-node', 'cluster_name': 'tdt4117-ir-data-cluster', 'cluster_uuid': '_J6siBOUQFuGHHFFIWmR4A', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [18]:
!mkdir enron && cd enron && wget https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz  && tar xf enron_mail_20150507.tar.gz

--2022-10-17 09:44:49--  https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 443254787 (423M) [application/x-gzip]
Saving to: ‘enron_mail_20150507.tar.gz’

enron_mail_20150507 100%[===================>] 422.72M  1022KB/s    in 7m 8s   

2022-10-17 09:51:58 (1011 KB/s) - ‘enron_mail_20150507.tar.gz’ saved [443254787/443254787]

tar: You may not specify more than one '-Acdtrux', '--delete' or  '--test-label' option
Try 'tar --help' or 'tar --usage' for more information.


In [20]:
from pathlib import Path
import shutil

files = Path("enron/maildir").glob("**/*")

# limit number of email to 100000
count = 0
for i, f in enumerate(files):
    if count >= 100000:
        break
    if f.is_file():
        with open(f, "r") as text:
            try:
                text.read()
                fn = str(f).replace('enron', 'enron_short')
                Path(fn).mkdir(parents=True, exist_ok=True)
                shutil.move(f, fn)
                count += 1
            except:
                print(f"skipping: {i} - {f}")

skipping: 12003 - enron/maildir/kitchen-l/_americas/regulatory/113.
skipping: 27229 - enron/maildir/lay-k/inbox/15.
skipping: 27477 - enron/maildir/lay-k/inbox/333.
skipping: 27861 - enron/maildir/lay-k/inbox/533.
skipping: 31141 - enron/maildir/skilling-j/all_documents/385.
skipping: 31973 - enron/maildir/skilling-j/deleted_items/346.
skipping: 33669 - enron/maildir/skilling-j/discussion_threads/306.
skipping: 34317 - enron/maildir/skilling-j/notes_inbox/100.
skipping: 50409 - enron/maildir/ring-a/inbox/9.
skipping: 51501 - enron/maildir/whalley-g/inbox/60.
skipping: 51693 - enron/maildir/whalley-g/inbox/390.
skipping: 94703 - enron/maildir/quigley-d/myfriends/142.
skipping: 94717 - enron/maildir/quigley-d/myfriends/44.
skipping: 98565 - enron/maildir/shankman-j/personal/12.


In [21]:
!rm -rf enron/maildir

In [22]:
from pathlib import Path

files = Path("enron_short/maildir").glob("**/*")

for i, f in enumerate(files):
    if f.is_file():
        with open(f, "r") as text:
            try:
                text = text.read()
                es.index(index="enron-index-000001", id=i, document={"text": text, "timestamp": datetime.now()})
            except:
                print(f"skipping: {i} - {f}")

In [26]:
es.search(index="enron-index-000001", query={"bool": {
    "filter": [{
      "query_string": {
        "query": "Norwegian and University and Science and Technology"
      }
    }]
}})

ObjectApiResponse({'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 0.0, 'hits': [{'_index': 'enron-index-000001', '_id': '4725', '_score': 0.0, '_ignored': ['text.keyword'], '_source': {'text': 'Message-ID: <20195007.1075841969571.JavaMail.evans@thyme>\nDate: Tue, 22 Jan 2002 09:36:09 -0800 (PST)\nFrom: d..hogan@enron.com\nTo: scott.neal@enron.com, frank.vickers@enron.com, andrea.ring@enron.com, \n\tf..brawner@enron.com, jonathan.mckay@enron.com, \n\tcraig.taylor@enron.com, chuck.ames@enron.com, \n\tjudy.townsend@enron.com, victoria.versen@enron.com, \n\tscott.goodell@enron.com, brad.mckay@enron.com, \n\tjared.kaiser@enron.com, bryant.frihart@enron.com, \n\tlaura.luce@enron.com, l..mims@enron.com, e.murrell@enron.com, \n\tpatrick.tucker@enron.com, h..lewis@enron.com, \n\tsouad.mahmassani@enron.com, denver.plachy@enron.com, \n\tkevin.ruscitti@enron.com, s..shively@enro

In [27]:
es.search(index="enron-index-000001", query={"bool": {
    "filter": [{
      "query_string": {
        "query": "Norwegian University Science Technology"
      }
    }]
}})

ObjectApiResponse({'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3122, 'relation': 'eq'}, 'max_score': 0.0, 'hits': [{'_index': 'enron-index-000001', '_id': '4742', '_score': 0.0, '_ignored': ['text.keyword'], '_source': {'text': 'Message-ID: <1406864.1075841968522.JavaMail.evans@thyme>\nDate: Mon, 4 Feb 2002 22:14:52 -0800 (PST)\nFrom: enerfaxtext@enerfax.com\nTo: judy.townsend@enron.com\nSubject: Natural Gas Futures End Choppy Day Down\nMime-Version: 1.0\nContent-Type: text/plain; charset=ANSI_X3.4-1968\nContent-Transfer-Encoding: 7bit\nX-From: "Enerfax" <Enerfaxtext@enerfax.com>@ENRON\nX-To: Townsend, Judy </O=ENRON/OU=NA/CN=RECIPIENTS/CN=JTOWNSE>\nX-cc: \nX-bcc: \nX-Folder: \\ExMerge - Townsend, Judy\\Deleted Items\nX-Origin: TOWNSEND-J\nX-FileName: judy townsend 6-26-02.PST\n\njudy.townsend@enron.com, To unsubscribe send a blank mailto:leave-enerfaxtext-14816594U@relay.netatlantic.com\n\n\n\nEnerfax 